Written for DragonFlight.

In [1]:
import numpy as np

In [3]:
# buff dict
buff_dict = {
    'rune': 1.4,
    'surge': 1.8,
    'touch': 1.2
}

In [83]:
# base powers for spells, in thousands
AB = 5.3  # check
AS = 54.5/buff_dict['surge']
AE = 6.2
missile = 3.15
arcane_echo = .7
barrage = 6
orb = 10.9
rs_dmg = 13

# tunable parameters
GCD = .9
nether_precision = True
tier_set = True
clearcast = True

In [4]:
# external conditions
n_targets = 0  # num secondary targets

Recall tier set, which is not implemented:

2-Set -  Mage Arcane Class Set 2pc - For each  Arcane Charge,  Arcane Blast critical strike chance is increased by 5% and  Arcane Explosion critical strike chance is increased by 3%.

4-Set -  Mage Arcane Class Set 4pc - When  Arcane Blast critically strikes at least one target, the critical strike chance of your next  Arcane Barrage is increased by 10%, stacking up to 4 times.

Assumptions made in calculations:

- Nether Precision does not time out before usage
- Clearcasting is either always on or off
- You're not going to cast missiles when RS is up
- Arcane Echo only does primary target damage bc idk how much it does
- You took ST-cleave target build: B4DAAAAAAAAAAAAAAAAAAAAAAAAAAAAAASIhWCSLJAAAAAAAAAAAAJJJJhkkkA

Note that Arcane Harmony starts at 20 stacks and Arcane Charges start at maximum.

Note that to simulate casting Touch of the Magi while Arcane Barrage is mid-air, you must write 'touch,barrage' instead of 'barrage,touch'.

In [84]:
def run_rotation(rotation_seq, verbose=False, verbose_lite=True):
    current_time = 0
    current_dmg = 0

    state_dict = {
        'RS':0,
        'nether_precision': 0,
        'harmony': 20,
        'charges': 4
    }
    time_dict = {
        'touch': 0
    }
    
    previous_spell = None
    for spell in rotation_seq.split(','):
        d = 0
        t = 0
        if spell == 'AB':
            if state_dict['nether_precision'] > 0:
                d = AB*1.2
                state_dict['nether_precision'] -= 1
            else:
                d = AB
            d *= 1 + state_dict['charges']*.7
            state_dict['charges'] += 1
            t = 1.4
        elif spell == 'AE':
            d = AE + AE*n_targets  # TODO: implement scaling
            t = GCD
        elif spell == 'AS':
            d = AS + (AS*min(n_targets, 5))
            t = 2.2
            time_dict['surge'] = 15 + t
            time_dict['rune'] = 15 + t
        elif spell == 'barrage':
            d = barrage
            d *= 1 + state_dict['charges']*.35
            d *= 1 + state_dict['harmony']*.05
            d *= 1 + .15*n_targets
            d += d*.4*n_targets  # non-primary dmg
            state_dict['charges'] = 0
            state_dict['harmony'] = 0
            t = GCD
        elif spell == 'M':
            if clearcast:
                n = 8
                state_dict['nether_precision'] = 2
            else:
                n = 5
            d = missile*n
            state_dict['harmony'] += n
            if time_dict['touch'] > 0:  # custom arcane echo
                d += arcane_echo*n
            t = 2.2
        elif spell == 'NT':  # TODO: implement (low prio bc it's relatively straightforward)
            raise NotImplementedError
        elif spell == 'orb':
            d = orb*(1+n_targets)  # TODO: implement scaling
            state_dict['charges'] += 2 + n_targets
            t = GCD
        elif spell == 'RS':
            d = rs_dmg
            state_dict['RS'] = 1
            t = 1.3
        elif spell == 'rune':
            t = 1.3
            time_dict['rune'] = 15 + t
        elif spell == 'touch':  # note that AOE damage was not considered here, but it's maximized when you maximize the rest
            state_dict['charges'] = 4
            t = 0
            time_dict['touch'] = 10
        else:
            raise ValueError(f'no spell named {spell}')
            
        # calculate arcane echo
        if time_dict['touch'] > 0 and spell in ['AB', 'AE', 'AS', 'barrage', 'orb', 'RS']:
            d += arcane_echo
            
        # pass the time
        for buff in time_dict.keys():
            if buff == 'touch' and spell == 'barrage' and previous_spell=='touch':  # because we cast touch while barrage is in midair
                state_dict['charges'] = 4
                continue
            time_dict[buff] = max(0, time_dict[buff]-t)
        
        # calculate buff percentage (after passing time)
        buff_perc = 1
        for buff in time_dict.keys():
            if time_dict[buff] > 0:
                buff_perc *= buff_dict[buff]
        if state_dict['RS'] > 0:
            rs_buff = 1 + (state_dict['RS']-1)/10*1.25  # radiant spark + harmonic echo
            buff_perc *= rs_buff
            if spell == 'orb':  # correction for RS only affecting primary target when spell is Arcane Orb
                buff_perc *= (n_targets/rs_buff + 1)/(n_targets + 1)
            
        current_dmg += d*buff_perc
        current_time += t

        # ensure maximum values
        state_dict['charges'] = min(4, state_dict['charges'])
        state_dict['harmony'] = min(20, state_dict['harmony'])
        
        if verbose:
            print(spell)
            print('dmg', d)
            print('buff', buff_perc)
            print('buffs', {**time_dict, **state_dict})
            
        # advance radiant spark
        if spell in ['AB', 'AE', 'AS', 'barrage', 'orb', 'RS', 'M']: # only update RS if the spell was an offensive spell
            if state_dict['RS'] > 0 and spell != 'RS':
                if state_dict['RS'] == 5:
                    state_dict['RS'] = 0
                else:
                    state_dict['RS'] += 1
        
        previous_spell = spell

    if verbose_lite:
        print('total dmg', current_dmg)
        print('total time', current_time)
    return current_dmg/current_time

## Full bursts

Multi-target rotations. Triple barrage, finally enabled by the instant-cast TotM, is the way to go here.

In [91]:
n_targets=2
run_rotation("rune,RS,AS,barrage,orb,touch,barrage,barrage,M,AB,AB")

total dmg 975.7670719999999
total time 13.400000000000002


72.8184382089552

In [96]:
run_rotation("rune,RS,AB,AB,AB,AS,touch,barrage,M,AB")  # ST rotation

total dmg 905.3043439999999
total time 13.500000000000002


67.05958103703702

In [92]:
run_rotation("rune,RS,touch,AB,AS,barrage,orb,barrage,M,AB,AB")

total dmg 1035.141296
total time 13.900000000000002


74.47059683453236

In [93]:
run_rotation("rune,RS,touch,AS,AB,barrage,orb,barrage,M,AB,AB")

total dmg 1036.428176
total time 13.900000000000002


74.56317812949638

Crabcore's multi-target scenario from guide:

In [179]:
n_targets=2
run_rotation("RS,orb,AS,touch,barrage,AB,barrage")  # guide

total dmg 803.273236
total time 7.600000000000001


105.69384684210524

In [178]:
run_rotation("RS,AS,touch,barrage,barrage,orb,barrage")  # me

total dmg 873.21292
total time 7.100000000000001


122.98773521126759

In [177]:
run_rotation("RS,AS,AB,AB,touch,barrage,barrage,orb,M,M,M,barrage")  # better if you have the time

total dmg 1387.9141000000002
total time 16.5


84.11600606060607

Single target rotations. As long as you use the touch-barrage tric, they're essentially the same.

In [101]:
n_targets=0
clearcast=True
run_rotation("rune,RS,AB,AB,AB,AS,touch,barrage,M,AB,AB,M,AB,AB,M,AB,AB,M,AB,AB,M,AB,AB")  # recommended

total dmg 1122.2816159999998
total time 34.89999999999999


32.15706636103152

In [114]:
run_rotation("RS,AS,AB,AB,AB,touch,barrage,M,AB,AB,M,AB,AB,rune,M,AB,AB,M,AB,AB,M,AB,AB")  # putting AS in front, rune in back

total dmg 1149.3678959999995
total time 34.89999999999999


32.93317753581661

In [103]:
run_rotation("RS,AS,AB,barrage,touch,barrage,barrage,M,AB,AB,M,AB,AB,M,AB,AB,M,AB,AB,M,AB,AB,AB")  # borrowed from multi-target rotation

total dmg 951.3078840000001
total time 33.999999999999986


27.979643647058836

In [104]:
run_rotation("RS,touch,AS,AB,AB,AB,barrage,M,AB,AB,M,AB,AB,M,AB,AB,M,AB,AB,M,AB,AB")  # "shortened"

total dmg 957.2843200000001
total time 33.59999999999999


28.490604761904777

In [106]:
run_rotation("RS,touch,AS,AB,AB,AB,barrage,M,AB,AB,M,AB,AB,rune,M,AB,AB,M,AB,AB,M,AB,AB")  # "shortened" with a rune

total dmg 1035.8603199999998
total time 34.89999999999999


29.68081146131805

## Mini bursts

In [169]:
n_targets=2
run_rotation("rune,RS,AB,AB,AB,touch,barrage,barrage,orb,M,AB,AB,barrage")  # triple barrage

total dmg 629.1734119999999
total time 15.400000000000004


40.855416363636344

In [163]:
run_rotation("rune,RS,touch,AB,AB,barrage,orb,M,AB,AB,barrage")  # touch in front

total dmg 526.91464
total time 13.100000000000003


40.22249160305342

In [164]:
run_rotation("rune,RS,AB,AB,AB,AB,touch,barrage,orb,M,AB,AB,barrage")  # standard

total dmg 595.564312
total time 15.900000000000004


37.45687496855345

## Misc

A few lingering questions:

In [32]:
clearcast=False
run_rotation("AB,AB,AB")/run_rotation("M,AB,AB")

1.2682349848863192

In [33]:
clearcast=True
run_rotation("AB,AB,AB")/run_rotation("M,AB,AB")

0.9637663790348359

In [34]:
clearcast=True
run_rotation("touch,M,M,M")/run_rotation("touch,M,AB,AB")

0.8914926133469179

Conclusion: clearcast missiles into 2x AB is best, even with missiles proccing arcane echo.